<a href="https://colab.research.google.com/github/santiagonajera/optimizacionInventrios/blob/main/Optimo_Inventario_K.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd

# Cargar el archivo desde GitHub
file_path = "https://github.com/santiagonajera/Optimizacion-Global-Inventarios/raw/refs/heads/main/Clases-Keisser.xlsx"
data_forecast = pd.read_excel(file_path, sheet_name='Forecast')
data_prices = pd.read_excel(file_path, sheet_name='Precios-Costos')

# Calcular las ventas totales por ítem
data_forecast['Total Sales'] = data_forecast.iloc[:, 1:].sum(axis=1)  # Asumiendo que las columnas de ventas inician desde la segunda

# Unir con los precios
merged_data = pd.merge(
    data_forecast[['ITEM', 'Total Sales']],
    data_prices[['ITEM', 'Precio']],
    on='ITEM'
)

# Calcular el valor total en dólares para cada ítem
merged_data['Total Value'] = merged_data['Total Sales'] * merged_data['Precio']

# Ordenar por valor total y calcular el porcentaje acumulado
merged_data = merged_data.sort_values(by='Total Value', ascending=False)
merged_data['Cumulative Percentage'] = merged_data['Total Value'].cumsum() / merged_data['Total Value'].sum() * 100

# Clasificación ABC con los valores por defecto como 'C'
bins = [0, 60, 80, 100]
labels = ['A', 'B', 'C']
merged_data['ABC Classification'] = pd.cut(
    merged_data['Cumulative Percentage'], bins=bins, labels=labels, include_lowest=True
)
merged_data['ABC Classification'] = merged_data['ABC Classification'].fillna('C')

# Mostrar el resultado
print(merged_data)



         ITEM  Total Sales  Precio  Total Value  Cumulative Percentage  \
1      ITEM 2       677477    4.17   2825079.09               0.720652   
6      ITEM 7       600608    4.35   2612644.80               1.387113   
120  ITEM 121       557675    4.24   2364542.00               1.990286   
62    ITEM 63       531327    4.20   2231573.40               2.559540   
3      ITEM 4       585719    3.66   2143731.54               3.106386   
..        ...          ...     ...          ...                    ...   
172  ITEM 173       243898    2.93    714621.14              99.291234   
88    ITEM 89       248826    2.84    706665.84              99.471498   
146  ITEM 147       237373    2.95    700250.35              99.650125   
116  ITEM 117       238808    2.88    687767.04              99.825568   
114  ITEM 115       234984    2.91    683803.44             100.000000   

    ABC Classification  
1                    A  
6                    A  
120                  A  
62         

In [5]:
import pandas as pd

# Cargar el archivo desde GitHub
file_path = "https://github.com/santiagonajera/Optimizacion-Global-Inventarios/raw/refs/heads/main/Clases-Keisser.xlsx"
data_forecast = pd.read_excel(file_path, sheet_name='Forecast')
data_prices = pd.read_excel(file_path, sheet_name='Precios-Costos')
data_historico = pd.read_excel(file_path, sheet_name='Historico')

# Calcular las ventas totales por ítem
data_forecast['Total Sales'] = data_forecast.iloc[:, 1:].sum(axis=1)

# Unir con los precios
merged_data = pd.merge(
    data_forecast[['ITEM', 'Total Sales']],
    data_prices[['ITEM', 'Precio']],
    on='ITEM'
)

# Calcular el valor total en dólares para cada ítem
merged_data['Total Value'] = merged_data['Total Sales'] * merged_data['Precio']

# Ordenar por valor total y calcular el porcentaje acumulado
merged_data = merged_data.sort_values(by='Total Value', ascending=False)
merged_data['Cumulative Percentage'] = merged_data['Total Value'].cumsum() / merged_data['Total Value'].sum() * 100

# Clasificación ABC con los valores por defecto como 'C'
bins = [0, 60, 80, 100]
labels = ['A', 'B', 'C']
merged_data['ABC Classification'] = pd.cut(
    merged_data['Cumulative Percentage'], bins=bins, labels=labels, include_lowest=True
)
merged_data['ABC Classification'] = merged_data['ABC Classification'].fillna('C')

# Calcular el coeficiente de variación para los últimos 18 meses en la hoja 'Historico'
historico_last_18 = data_historico.iloc[:, -18:]  # Últimos 18 meses
historico_last_18['Mean'] = historico_last_18.mean(axis=1)
historico_last_18['StdDev'] = historico_last_18.std(axis=1)
historico_last_18['Coefficient of Variation'] = historico_last_18['StdDev'] / historico_last_18['Mean']

# Agregar los coeficientes de variación al dataframe de ítems
data_historico['Coefficient of Variation'] = historico_last_18['Coefficient of Variation']

# Clasificar en X, Y, Z basado en percentiles
percentiles = data_historico['Coefficient of Variation'].quantile([0.33, 0.67])
data_historico['XYZ Classification'] = pd.cut(
    data_historico['Coefficient of Variation'],
    bins=[-float('inf'), percentiles[0.33], percentiles[0.67], float('inf')],
    labels=['X', 'Y', 'Z']
)

# Unir las clasificaciones ABC y XYZ para cada ítem
final_classification = pd.merge(
    merged_data[['ITEM', 'ABC Classification']],
    data_historico[['ITEM', 'Coefficient of Variation', 'XYZ Classification']],
    on='ITEM'
)

# Mostrar el resultado final
print(final_classification)


         ITEM ABC Classification  Coefficient of Variation XYZ Classification
0      ITEM 2                  A                  0.204201                  Y
1      ITEM 7                  A                  0.213136                  Z
2    ITEM 121                  A                  0.207599                  Y
3     ITEM 63                  A                  0.212072                  Z
4      ITEM 4                  A                  0.204985                  Y
..        ...                ...                       ...                ...
295  ITEM 173                  C                  0.190998                  X
296   ITEM 89                  C                  0.170195                  X
297  ITEM 147                  C                  0.196177                  X
298  ITEM 117                  C                  0.218294                  Z
299  ITEM 115                  C                  0.182495                  X

[300 rows x 4 columns]


In [6]:
import pandas as pd

# Cargar el archivo desde GitHub
file_path = "https://github.com/santiagonajera/Optimizacion-Global-Inventarios/raw/refs/heads/main/Clases-Keisser.xlsx"
data_forecast = pd.read_excel(file_path, sheet_name='Forecast')
data_prices = pd.read_excel(file_path, sheet_name='Precios-Costos')
data_historico = pd.read_excel(file_path, sheet_name='Historico')

# Calcular las ventas totales por ítem
data_forecast['Total Sales'] = data_forecast.iloc[:, 1:].sum(axis=1)

# Unir con los precios
merged_data = pd.merge(
    data_forecast[['ITEM', 'Total Sales']],
    data_prices[['ITEM', 'Precio']],
    on='ITEM'
)

# Calcular el valor total en dólares para cada ítem
merged_data['Total Value'] = merged_data['Total Sales'] * merged_data['Precio']

# Ordenar por valor total y calcular el porcentaje acumulado
merged_data = merged_data.sort_values(by='Total Value', ascending=False)
merged_data['Cumulative Percentage'] = merged_data['Total Value'].cumsum() / merged_data['Total Value'].sum() * 100

# Clasificación ABC con los valores por defecto como 'C'
bins = [0, 60, 80, 100]
labels = ['A', 'B', 'C']
merged_data['ABC Classification'] = pd.cut(
    merged_data['Cumulative Percentage'], bins=bins, labels=labels, include_lowest=True
)
merged_data['ABC Classification'] = merged_data['ABC Classification'].fillna('C')

# Calcular el coeficiente de variación para los últimos 18 meses en la hoja 'Historico'
historico_last_18 = data_historico.iloc[:, -18:]  # Últimos 18 meses
historico_last_18['Mean'] = historico_last_18.mean(axis=1)
historico_last_18['StdDev'] = historico_last_18.std(axis=1)
historico_last_18['Coefficient of Variation'] = historico_last_18['StdDev'] / historico_last_18['Mean']

# Agregar los coeficientes de variación al dataframe de ítems
data_historico['Coefficient of Variation'] = historico_last_18['Coefficient of Variation']

# Clasificar en X, Y, Z basado en percentiles
percentiles = data_historico['Coefficient of Variation'].quantile([0.33, 0.67])
data_historico['XYZ Classification'] = pd.cut(
    data_historico['Coefficient of Variation'],
    bins=[-float('inf'), percentiles[0.33], percentiles[0.67], float('inf')],
    labels=['X', 'Y', 'Z']
)

# Unir las clasificaciones ABC y XYZ para cada ítem
final_classification = pd.merge(
    merged_data[['ITEM', 'ABC Classification']],
    data_historico[['ITEM', 'Coefficient of Variation', 'XYZ Classification']],
    on='ITEM'
)

# Define the service level matrix as a dictionary
service_level_matrix = {
    'A': {'X': 0.95, 'Y': 0.90, 'Z': 0.85},
    'B': {'X': 0.85, 'Y': 0.80, 'Z': 0.75},
    'C': {'X': 0.75, 'Y': 0.70, 'Z': 0.65}
}

# Assign service levels based on ABC and XYZ classifications
def assign_service_level(row):
    return service_level_matrix[row['ABC Classification']][row['XYZ Classification']]

# Apply the function to assign service levels to each item
final_classification['Service Level'] = final_classification.apply(assign_service_level, axis=1)

# Mostrar el resultado final
print(final_classification)


         ITEM ABC Classification  Coefficient of Variation XYZ Classification  \
0      ITEM 2                  A                  0.204201                  Y   
1      ITEM 7                  A                  0.213136                  Z   
2    ITEM 121                  A                  0.207599                  Y   
3     ITEM 63                  A                  0.212072                  Z   
4      ITEM 4                  A                  0.204985                  Y   
..        ...                ...                       ...                ...   
295  ITEM 173                  C                  0.190998                  X   
296   ITEM 89                  C                  0.170195                  X   
297  ITEM 147                  C                  0.196177                  X   
298  ITEM 117                  C                  0.218294                  Z   
299  ITEM 115                  C                  0.182495                  X   

     Service Level  
0     

In [10]:
import pandas as pd
from scipy.stats import norm

# Cargar el archivo desde GitHub
file_path = "https://github.com/santiagonajera/Optimizacion-Global-Inventarios/raw/refs/heads/main/Clases-Keisser.xlsx"
data_forecast = pd.read_excel(file_path, sheet_name='Forecast')
data_prices = pd.read_excel(file_path, sheet_name='Precios-Costos')
data_historico = pd.read_excel(file_path, sheet_name='Historico')
data_leadtime = pd.read_excel(file_path, sheet_name='LeadTime-Dias')

# Calcular la media y desviación estándar de las ventas de los últimos 18 meses
historico_last_18 = data_historico.iloc[:, :].copy()  # Aseguramos incluir todas las columnas
historico_last_18['Mean'] = historico_last_18.iloc[:, 1:].mean(axis=1)
historico_last_18['StdDev'] = historico_last_18.iloc[:, 1:].std(axis=1)
historico_last_18['ITEM'] = data_historico['ITEM']  # Restaurar la columna ITEM si no está incluida

# Calcular el Z de alfa basado en el nivel de servicio
def calculate_z(service_level):
    return norm.ppf(service_level)

# Agregar los niveles de servicio al dataset
final_classification['Z Alpha'] = final_classification['Service Level'].apply(calculate_z)

# Calcular el Lead Time Promedio
data_leadtime['LeadTime Promedio'] = data_leadtime.iloc[:, 1:].mean(axis=1)

# Calcular el factor de lead time
data_leadtime['LeadTime Factor'] = (data_leadtime['LeadTime Promedio'] + 0.5) ** 0.5

# Unir todos los datos necesarios
final_data = pd.merge(
    final_classification,
    historico_last_18[['ITEM', 'StdDev']],
    on='ITEM'
)
final_data = pd.merge(
    final_data,
    data_leadtime[['ITEM', 'LeadTime Factor']],
    on='ITEM'
)

# Calcular el stock de seguridad
final_data['Safety Stock'] = final_data['Z Alpha'] * final_data['StdDev'] * final_data['LeadTime Factor']

# Mostrar el resultado final
print(final_data)


         ITEM ABC Classification  Coefficient of Variation XYZ Classification  \
0      ITEM 2                  A                  0.204201                  Y   
1      ITEM 7                  A                  0.213136                  Z   
2    ITEM 121                  A                  0.207599                  Y   
3     ITEM 63                  A                  0.212072                  Z   
4      ITEM 4                  A                  0.204985                  Y   
..        ...                ...                       ...                ...   
295  ITEM 173                  C                  0.190998                  X   
296   ITEM 89                  C                  0.170195                  X   
297  ITEM 147                  C                  0.196177                  X   
298  ITEM 117                  C                  0.218294                  Z   
299  ITEM 115                  C                  0.182495                  X   

     Service Level   Z Alph